In [1]:
import numpy as np
import xarray as xr
import intake

In [2]:
from pyrte_rrtmgp import rrtmgp_cloud_optics, rrtmgp_gas_optics
from pyrte_rrtmgp.data_types import (
    CloudOpticsFiles,
    GasOpticsFiles,
    OpticsProblemTypes,
)
from pyrte_rrtmgp.rte_solver import rte_solve

In [3]:
import warnings

# Suppress specific FutureWarnings matching the message pattern when using cat[...].to_dask()
warnings.filterwarnings(
    "ignore",
    message=".*The return type of `Dataset.dims` will be changed.*",
    category=FutureWarning,
)

# Read data

Zoom level 5 is 12288 points (roughly 5 degrees); each zoom level (max 11, min 0) is 4x more, fewer points or 2x higher in grid density

Perhaps chunks should be introduced at this stage? 

In [4]:
cat = intake.open_catalog('https://digital-earths-global-hackathon.github.io/catalog/catalog.yaml')['online']

In [5]:
zoom = 5
data = cat["icon_d3hp003feb"](zoom=zoom).to_dask()

In [6]:
# Local copy
# data = xr.open_dataset("/Users/robert/Codes/hk25/data/PT15M_inst_z5_atm", consolidated=True, engine='zarr')

In [7]:
data

<xarray.Dataset> Size: 1GB
Dimensions:   (crs: 1, time: 96, pressure: 30, cell: 12288)
Coordinates:
  * crs       (crs) float32 4B nan
  * pressure  (pressure) int64 240B 5 10 20 50 100 ... 92500 95000 97500 100000
  * time      (time) datetime64[ns] 768B 2020-02-01T00:15:00 ... 2020-02-02
Dimensions without coordinates: cell
Data variables: (12/18)
    hur       (time, pressure, cell) float32 142MB ...
    hus       (time, pressure, cell) float32 142MB ...
    orog      (cell) float32 49kB ...
    pr        (time, cell) float32 5MB ...
    psl       (time, cell) float32 5MB ...
    qall      (time, pressure, cell) float32 142MB ...
    ...        ...
    ua        (time, pressure, cell) float32 142MB ...
    uas       (time, cell) float32 5MB ...
    va        (time, pressure, cell) float32 142MB ...
    vas       (time, cell) float32 5MB ...
    wa        (time, pressure, cell) float32 142MB ...
    zg        (time, pressure, cell) float32 142MB ...

# Transform the data to the form needed to compute fluxes 

## Pressure values on levels
 Top level pressure is arbitrary, bottom level presure would normally be surface pressure but these data have been 
 interpolated to fixed pressures so the surface pressure can be well below 1000 hPa 

In [8]:
data["pressure_h"] = xr.concat([
                        xr.DataArray([1], dims="pressure_h"),
                        xr.DataArray(((data.pressure.values[1:] * data.pressure.values[:-1]) ** 0.5), dims="pressure_h"),
                        xr.DataArray([100500], dims="pressure_h")
                    ], dim = "pressure_h")

## Temperature on levels
Linear interpolation of temperature in pressure
  Temperature at top level is same as top layer 
  Temperature at bottom level is surface T (might be colder than lowest layer when elevation is high) 


In [9]:
data["ta_h"] = data.ta.interp(pressure=data.pressure_h)
data["ta_h"][:,  0, :] = data.ta[:,  0, :]
data["ta_h"][:, -1, :] = data.ts

## Water vapor - convert specific humidity to molar mixing ratio 

Molar mixing ratio assuming specific humidity is water mass/dry air mass (not quite correct)
  Md, Mw are molar mases of dry air and water vapor 


In [10]:
Md = 0.0289652
Mw = 0.018016
data["h2o"] = data.hus * (Md/Mw)

## Ozone from monthly-mean ERA5 interpolated onto HEALPix grid at zoom levels 8 and below

In [11]:
Mo3 = .047998
if zoom <= 8:
    data["o3"] = cat["ERA5"](zoom=zoom)\
        .to_dask()\
        .sel(time="2020-02-01", method="nearest")\
        .o3.interp(level=data.pressure)\
        .reset_coords(("lat", "lon", "level", "time"))\
        .drop_vars(("lat", "lon", "level", "time"))\
        .o3 * (Md/Mo3)

data.o3.attrs['units'] = "1"   
# This is actually a mass fraction; need to set to vmr 
# also need to change/delete units
    
# For zoom > 8 we need to interpolate in space too - probably nearest neighbor 

## Well-mixed greenhouse gases (these are pre-industrial values, should update) 

In [12]:
gas_values = {
        "co2": 348e-6,
        "ch4": 1650e-9,
        "n2o": 306e-9,
        "n2": 0.7808,
        "o2": 0.2095,
        "co": 0.0,
}
for gas_name, value in gas_values.items():
    data[gas_name] = value

## Cloud properties 
  Data set includes only `qall` all hydrometeors 
  - Assume all clouds > 263 are liquid, everything else is ice (could refine)
  - Convert from MMR to vertically-integrated LWP, IWP (haven't done this yet)

In [13]:
data["lwp"] = xr.where(data.ta >= 263., data.qall, 0)  
data["iwp"] = xr.where(data.ta <  263., data.qall, 0)  

# Liquid and ice effective sizes in microns 
data["rel"] = xr.where(data.lwp > 0., 10., 0)  
data["rei"] = xr.where(data.iwp > 0,  35., 0)  

In [20]:
# ## Change variable and coordinate names to those needed by pyRTE 

# Workaround
#    top_at_1 determination assumes 2D pressure arrays
#    we add this array and drop the 1D pressure variable
#    need to revise to use isel(layer=0)[0] and (layer=-1)[0]
data["p2"] = data["pressure"].broadcast_like(data.ta)

var_mapping = {"p2":"pres_layer", 
               "pressure_h":"pres_level", 
               "ta":"temp_layer", 
               "ta_h":"temp_level"}

atmosphere = data.rename_dims({"pressure":"layer", 
                               "pressure_h":"level"})\
                 .rename(var_mapping)\
                 .isel(time=6)\
                 .drop_vars(("pressure", "crs"))
atmosphere

<xarray.Dataset> Size: 30MB
Dimensions:     (layer: 30, cell: 12288, level: 31)
Coordinates:
    time        datetime64[ns] 8B 2020-02-01T01:45:00
  * pres_level  (level) float64 248B 1.0 7.071 14.14 ... 9.874e+04 1.005e+05
  * cell        (cell) float32 49kB 0.0 1.0 2.0 ... 1.229e+04 1.229e+04
Dimensions without coordinates: layer, level
Data variables: (12/32)
    hur         (layer, cell) float32 1MB ...
    hus         (layer, cell) float32 1MB 4.415e-06 4.419e-06 ... 0.01866
    orog        (cell) float32 49kB ...
    pr          (cell) float32 49kB ...
    psl         (cell) float32 49kB ...
    qall        (layer, cell) float32 1MB 0.0 0.0 0.0 ... 5.959e-05 6.546e-05
    ...          ...
    co          float64 8B 0.0
    lwp         (layer, cell) float32 1MB 0.0 0.0 0.0 ... 5.959e-05 6.546e-05
    iwp         (layer, cell) float32 1MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    rel         (layer, cell) float64 3MB 0.0 0.0 0.0 0.0 ... 10.0 10.0 10.0
    rei         (layer, cell) float64 3MB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    pres_layer  (layer, cell) int64 3MB 5 5 5 5 ... 100000 100000 100000 100000

# pyRTE

## Initialization 

In [16]:
cloud_optics_lw = rrtmgp_cloud_optics.load_cloud_optics(
    cloud_optics_file=CloudOpticsFiles.LW_BND
)
gas_optics_lw = rrtmgp_gas_optics.load_gas_optics(
    gas_optics_file=GasOpticsFiles.LW_G128
)

cloud_optics_sw = rrtmgp_cloud_optics.load_cloud_optics(
    cloud_optics_file=CloudOpticsFiles.SW_BND
)
gas_optics_sw = rrtmgp_gas_optics.load_gas_optics(
    gas_optics_file=GasOpticsFiles.SW_G112
)



## Test gas optics

In [21]:
# 
# SW gas optics
#   Not clear that this is running in parallel 
#   And there are some NaNs in the tau field... that's bad
#
sw_optics = gas_optics_sw.compute_gas_optics(
                atmosphere,
                problem_type=OpticsProblemTypes.TWO_STREAM, 
                add_to_input=False,
)

sw_optics["tau"].values

array([[[7.71711064e-07, 7.81976343e-07, 7.16109278e-07, ...,
         5.40283718e-07, 5.80230131e-07, 5.67599689e-07],
        [3.42751777e-06, 3.48712762e-06, 3.11060242e-06, ...,
         2.11292527e-06, 2.33664034e-06, 2.26848863e-06],
        [6.40741416e-06, 6.46791985e-06, 6.07939178e-06, ...,
         5.00575220e-06, 5.25612554e-06, 5.17544410e-06],
        ...,
        [1.17122929e-02, 1.16928333e-02, 1.16743988e-02, ...,
         1.17338183e-02, 1.17387684e-02, 1.16815098e-02],
        [5.50791358e-02, 5.49694713e-02, 5.48736018e-02, ...,
         5.52224248e-02, 5.52462155e-02, 5.49284071e-02],
        [1.10615322e-01, 1.10391872e-01, 1.10198916e-01, ...,
         1.10913866e-01, 1.10961190e-01, 1.10315009e-01]],

       [[3.90783088e-07, 4.02153160e-07, 4.54060370e-07, ...,
         1.62587326e-07, 2.08206462e-07, 1.86287868e-07],
        [1.48286372e-06, 1.52912966e-06, 1.73825448e-06, ...,
         6.50077165e-07, 7.74661527e-07, 7.14808932e-07],
        [4.08422721e-06, 

In [22]:
# 
# LW gas optics
#   This doesn't work full stop 
#   
#
lw_optics = gas_optics_lw.compute_gas_optics(
                atmosphere,
                problem_type=OpticsProblemTypes.ABSORPTION, 
                add_to_input=False,
)

lw_optics["tau"].values

AlignmentError: cannot align objects with join='exact' where index/labels/sizes are not equal along these coordinates (dimensions): 'minor_absorber_intervals_lower' ('minor_absorber_intervals_lower',)

## Test cloud optics

In [23]:
# 
# Shortwave cloud optics
#
sw_cld_optics = cloud_optics_lw.compute_cloud_optics(
    atmosphere, 
    problem_type=OpticsProblemTypes.TWO_STREAM
)
sw_cld_optics["tau"]

<xarray.DataArray 'tau' (layer: 30, bnd: 16, cell: 12288)> Size: 47MB
array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
...
         7.19043902e-06, 6.24950998e-06, 1.21311063e-05],
        [1.23120534e-06, 1.31353766e-07, 8.84573332e-07, ...,
         7.07688096e-06, 6.15081194e-06, 1.19395207e-05],
        [1.20374382e-06, 1.28423974e-07, 8.64843291e-07, ...,
         6.91903406e-06, 6.01362063e-06, 1.16732146e-05]],

       [[4.51457069e-09, 1.64375244e-08, 1.15693802e-07, ...,
         5.70098337e-06, 6.82266106e-06, 7.49535751e-06],
        [6.81166484e-09, 2.48012302e-08, 1.74560874e-07, ...,
         8.60174546e-06, 1.02941528e-05, 1.13091292e-05],
        [7.07137888e-09, 2.57468474e-08, 1.81216502e-07, ...,
         8.92971141e-06, 1.06866466e-05, 1.17403218e-05],
        ...,
        [7.10721365e-09, 2.58773216e-08, 1.82134831e-07, ...,
         8.97496342e-06, 1.07408020e-05, 1.17998168e-05],
        [6.99496997e-09, 2.54686431e-08, 1.79258390e-07, ...,
         8.83322251e-06, 1.05711733e-05, 1.16134632e-05],
        [6.83895007e-09, 2.49005755e-08, 1.75260106e-07, ...,
         8.63620115e-06, 1.03353877e-05, 1.13544297e-05]]],
      shape=(30, 16, 12288))
Coordinates:
  * cell     (cell) float32 49kB 0.0 1.0 2.0 ... 1.228e+04 1.229e+04 1.229e+04
    time     datetime64[ns] 8B 2020-02-01T01:45:00
Dimensions without coordinates: layer, bnd

In [24]:
# 
# Longwave cloud optics
#
lw_cld_optics = cloud_optics_lw.compute_cloud_optics(
    atmosphere, 
    problem_type=OpticsProblemTypes.ABSORPTION
)
lw_cld_optics["tau"]

<xarray.DataArray 'tau' (layer: 30, bnd: 16, cell: 12288)> Size: 47MB
array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
...
         1.29789246e-06, 1.12805239e-06, 2.18969543e-06],
        [1.41206177e-07, 1.50648819e-08, 1.01451167e-07, ...,
         8.11643089e-07, 7.05432808e-07, 1.36933622e-06],
        [4.05892677e-07, 4.33035251e-08, 2.91618161e-07, ...,
         2.33304231e-06, 2.02774422e-06, 3.93611353e-06]],

       [[3.06262695e-09, 1.11510061e-08, 7.84851941e-08, ...,
         3.86747412e-06, 4.62840590e-06, 5.08475456e-06],
        [3.89078458e-09, 1.41663230e-08, 9.97081877e-08, ...,
         4.91326854e-06, 5.87996207e-06, 6.45971088e-06],
        [4.02411767e-09, 1.46517879e-08, 1.03125082e-07, ...,
         5.08164108e-06, 6.08146218e-06, 6.68107837e-06],
        ...,
        [1.28287007e-09, 4.67092213e-09, 3.28757986e-08, ...,
         1.62000364e-06, 1.93874197e-06, 2.12989684e-06],
        [8.02248768e-10, 2.92098288e-09, 2.05590336e-08, ...,
         1.01307681e-06, 1.21240131e-06, 1.33194090e-06],
        [2.30603863e-09, 8.39627262e-09, 5.90962898e-08, ...,
         2.91205716e-06, 3.48500911e-06, 3.82862185e-06]]],
      shape=(30, 16, 12288))
Coordinates:
  * cell     (cell) float32 49kB 0.0 1.0 2.0 ... 1.228e+04 1.229e+04 1.229e+04
    time     datetime64[ns] 8B 2020-02-01T01:45:00
Dimensions without coordinates: layer, bnd

## Compute fluxes from atmosphere conditions  

In [32]:
#
# Shortwave fluxes 
# 
sw_fluxes = rte_solve(
    xr.merge(
        [cloud_optics_sw.compute_cloud_optics(
            atmosphere, 
            problem_type=OpticsProblemTypes.TWO_STREAM, 
         )\
         .add_to(
             gas_optics_sw.compute_gas_optics(
                    atmosphere,
                    problem_type=OpticsProblemTypes.TWO_STREAM, 
                    add_to_input=False,
             ),
             delta_scale = True,
         ), 
         xr.Dataset(data_vars = {"surface_albedo":0.06, 
                                "mu0":0.86}
                   ),
        ],
    ),
    add_to_input = False,
)

In [30]:
#
# Longwave fluxes 
# 
lw_fluxes = rte_solve(
    xr.merge(
        [cloud_optics_lw.compute_cloud_optics(
            atmosphere, 
            problem_type=OpticsProblemTypes.ABSORPTION, 
         )\
         .add_to(
             gas_optics_lw.compute_gas_optics(
                    atmosphere,
                    problem_type=OpticsProblemTypes.ABSORPTION, 
                    add_to_input=False,
             ), 
         ), 
        xr.Dataset(data_vars = {"surface_emissivity":0.98}),
        ],
    ),
    add_to_input = False,
)

AlignmentError: cannot align objects with join='exact' where index/labels/sizes are not equal along these coordinates (dimensions): 'minor_absorber_intervals_lower' ('minor_absorber_intervals_lower',)